In [ ]:
using NamedTrajectories
using QuantumCollocation

using CairoMakie
using ForwardDiff
using LinearAlgebra
using Random

# Example III.1
-----
**How to build a quantum control problem**

Every quantum control problem requires a quantum system describing the dynamics, a goal, and time.

In [ ]:
system = QuantumSystem(0.01 * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])
U_goal = GATES[:X]

## Number of timesteps
T = 50

## Duration of timestep
Δt = 0.2

**NamedTrajectories..jl** stores the problem data. It has a lot of fields to help with this.

In [ ]:
NamedTrajectory |> fieldnames

In [ ]:
n_drives = length(system.H_drives)
a_bounds = fill(1.0, n_drives)
da_bounds = fill(1.0, n_drives)
dda_bounds = fill(1.0, n_drives)

## This will help us initialize the trajectory easily
traj = initialize_unitary_trajectory(
    U_goal,
    T,
    Δt,
    n_drives,
    (a = a_bounds, da = da_bounds, dda = dda_bounds)
)

The main thing a NamedTrajectory supports is indexing by symbols and plotting.

In [ ]:
## Inspect the control
traj.a

In [ ]:
plot(traj, [:a, :Ũ⃗])

Trajectories also enable simple construction of objectives.

In [ ]:
Q = 100.0
## Notice that we are using the iso_vec_operator symbol.
J = UnitaryInfidelityObjective(:Ũ⃗, traj, Q)

## Loss functions are evaluated on trajectories
Z⃗ = vec(traj)
J.L(Z⃗, traj)

Look at that! The infidelity loss is already zero. Are we done?

In [ ]:
R = 1e-2
J += QuadraticRegularizer(:a, traj, R)
J += QuadraticRegularizer(:da, traj, R)
J += QuadraticRegularizer(:dda, traj, R)

J.L(Z⃗, traj)

Let's add the dynamics constraints.

In [ ]:
## Integrators
integrators = [
    UnitaryPadeIntegrator(system, :Ũ⃗, :a, traj),
    DerivativeIntegrator(:a, :da, traj),
    DerivativeIntegrator(:da, :dda, traj)
]

In [ ]:
Random.seed!(1234)
ipopt_options = IpoptOptions(print_level=4, max_iter=50, recalc_y="yes", recalc_y_feas_tol=1e-2)

prob = QuantumControlProblem(
    system,
    traj,
    J,
    integrators,
    ipopt_options=ipopt_options,
)

In [ ]:
solve!(prob)
println("Unitary fidelity: ", unitary_fidelity(prob))

In [ ]:
plot(prob.trajectory, [:a, :Ũ⃗])

Of course, it is much easier to just call
```Julia
system = QuantumSystem(0.01 * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])
U_goal = GATES[:X]
T = 50
Δt = 0.2
problem = UnitarySmoothPulseProblem(system, U_goal, T, Δt)
solve!(problem)
```

# Exercises
-----

## Exercise III.1
**Inspect a gradient for correctness**

It is often the case that after we work very hard to solve for an analytic gradient, we need to make sure we did a good job coding it up. Not only do we need to profile the calculation to make sure our code is appropriately efficient, but we also need to be sure that our gradient was correct to begin with.

The next cell defines the following objective:
\begin{equation}
    J(\vec{\mathbf{Z}}) = ...
\end{equation}
Unfortunately, we made a mistake when writing our gradient.

Create a test trajectory, then use **ForwardDiff.jl** to help find the and fix the bug.

## Exercise III.2
**The problem template grand tour**

Constrained optimization gives **Piccolo.jl** the ability to solve many different kinds of problems. These next few examples will walk you through implementing many of these types.

In [ ]:
## This should be familiar by now.
Δ = 0.1
system = system = QuantumSystem(Δ * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])
U_goal = GATES[:X]

T = 50
Δt = 0.2

## Make sure to pass any provided options dictionaries to each problem! 
## ipopt_options = ...
## piccolo_options = ...

## Anything enclosed by `# ----- #` should remain untouched. Use the problem template to pass the test.

### Smooth pulse problems

In [ ]:
# ----- #
Random.seed!(42);
ipopt_options = IpoptOptions(print_level=1, max_iter=50)
piccolo_options = PiccoloOptions(verbose=false)
test_fidelity() = unitary_fidelity(smooth_pulse_prob) > 0.99 ? "Pass" : "Fail"
# ----- #

# smooth_pulse_prob = # TODO: UnitarySmoothPulseProblem
smooth_pulse_prob = UnitarySmoothPulseProblem(
    system, U_goal, T, Δt, 
    ipopt_options=ipopt_options,
    piccolo_options=piccolo_options
)

# ----- #
solve!(smooth_pulse_prob)
test_fidelity(smooth_pulse_prob)
# ----- #

In [ ]:
## View your result
plot_unitary_populations(smooth_pulse_prob.trajectory)

### Minimum time problems

In [ ]:
# ----- #
Random.seed!(42);
ipopt_options = IpoptOptions(print_level=1, max_iter=50, recalc_y="yes", recalc_y_feas_tol=1e-2)
piccolo_options = PiccoloOptions(verbose=false)
test_time() = get_duration(mintime_prob.trajectory) < get_duration(smooth_pulse_prob.trajectory) ? "Pass" : "Fail"
# ----- #

# mintime_prob = # TODO: UnitaryMinimumTimeProblem

mintime_prob = UnitaryMinimumTimeProblem(
    smooth_pulse_prob, final_fidelity=0.9999,
    ipopt_options=ipopt_options,
    piccolo_options=piccolo_options,
)

# ----- #
solve!(mintime_prob)
test_time()
# ----- #

In [ ]:
plot_unitary_populations(mintime_prob.trajectory)

### Bang-bang control

In [ ]:
# ----- #
Random.seed!(42)
piccolo_options = piccolo_options=PiccoloOptions(verbose=false)
ipopt_options = ipopt_options=IpoptOptions(print_level=1, max_iter=50)
function test_sparsity()
    r = 1e-3
    if 2sum(bangbang_prob.trajectory.da .> r) < sum(smooth_pulse_prob.trajectory.da .> r)
        "Pass"
    else
        "Fail. Try increasing the bang bang regularization parameter."
    end
end
# ----- #

# bangbang_prob = # TODO: UnitaryBangBangProblem

bangbang_prob = UnitaryBangBangProblem(
    system, U_goal, T, Δt, R_bang_bang=10.0,
    ipopt_options=ipopt_options,
    piccolo_options=piccolo_options
);

# ----- #
solve!(bangbang_prob)
test_sparsity()
# ----- #

In [ ]:
plot_unitary_populations(bangbang_prob.trajectory)

In [ ]:
plot(bangbang_prob.trajectory, [:da])

### Sampling over quantum systems
*Solving problems with shared controls.*

In [ ]:
sampleable_system(Δ) = QuantumSystem(Δ * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])

In [ ]:
## It doesn't work if the drift changes
unitary_fidelity(smooth_pulse_prob.trajectory, sampleable_system(Δ)) |> println
unitary_fidelity(smooth_pulse_prob.trajectory, sampleable_system(0.0)) |> println

Solve the problem for a bunch of different systems--but make everyone use the same controls!

In [ ]:
# ----- # 
Random.seed!(42)
ipopt_options = IpoptOptions(print_level=1, max_iter=50)
piccolo_options = PiccoloOptions(verbose=false)
function test_robustness()

end
# ----- #

# sample_prob = # TODO: UnitarySamplingProblem(

sample_prob = UnitarySamplingProblem(
    [sampleable_system(-0.1),  
     sampleable_system(0), 
     sampleable_system(0.1)],
    U_goal,
    T,
    Δt,
    ipopt_options=ipopt_options,
    piccolo_options=piccolo_options
);

# ----- #
solve!(sample_prob)
# ----- #

In [ ]:
unitary_fidelity(sample_prob.trajectory, sampleable_system(-0.1))

In [ ]:
sweep_sample(Δ) = unitary_fidelity(sample_prob.trajectory, sampleable_system(Δ), unitary_name=:Ũ⃗1)
sweep_default(Δ) = unitary_fidelity(smooth_pulse_prob.trajectory, sampleable_system(Δ))
     
f = Figure()
ax = Axis(f[1,1], yscale=log10, limits=(nothing, (10^-6, 1)))

Δs = range(-.2, .2, length=500)
lines!(ax, Δs, 1 .- sweep_sample.(Δs), label="Robust infidelity")
lines!(ax, Δs, 1 .- sweep_default.(Δs), label="Default infidelity")
axislegend(ax, position=:lb)
f

### Sampling over gates
*Solving independent problems, together.*

In [ ]:
sampleable_U_goal(θ) = exp(-im * θ * PAULIS[:X])

In [ ]:
# ----- #
Random.seed!(42)
Q_symb = :a
ipopt_options = IpoptOptions(print_level=1, max_iter=50)
piccolo_options = PiccoloOptions(verbose=false, free_time=false)
test_direct_sum() = nothing
# ----- #

θs = range(0, π, length=5)
probs = QuantumControlProblem[]
for θ in θs
    prob_θ = UnitarySmoothPulseProblem(
        system, sampleable_U_goal(θ), T, Δt,
        piccolo_options=piccolo_options,
        ipopt_options=ipopt_options
    )
    solve!(prob_θ)
    push!(probs, prob_θ)
end

direct_sum_prob = UnitaryDirectSumProblem(
    probs, 0.9999, Q_symb=Q_symb, 
    ipopt_options=ipopt_options, piccolo_options=piccolo_options
)

# ----- #
solve!(direct_sum_prob)
# ----- #

In [ ]:
control_symbols = [Symbol("a$i") for i in eachindex(θs)]
plot(direct_sum_prob.trajectory, control_symbols, merge_components=true, ignored_lables=control_symbols)

### EmbeddedOperators

Gates are defined in a computational subspace, but the physical system might not care. We need to model this, and adapt our costs.

In [ ]:
# ----- #
a = annihilate(4)
TRANSMON = Dict(
    :I => Matrix{ComplexF64}(I, size(a)),
    :X => (a + a') / 2,
    :Y => (a - a') / 2im,
    :N => a'a,
    :A => a'a'a*a
);
transmon = QuantumSystem(0.1 * (TRANSMON[:A]), [TRANSMON[:X], TRANSMON[:Y]])
test_embedding() = unembed(U_goal_embedded) == GATES[:X] ? "Pass" : "Fail"
# ----- #

# U_goal_embedded = # TODO: create an EmbeddedOperator for GATES[:X]
U_goal_embedded = EmbeddedOperator(GATES[:X], transmon)

# ----- #
test_embedding()
# ----- #

Penalizing leakage can help us avoid populating higher levels.